# 3.3 线性回归的简洁实现

## 3.3.1 生成数据集

In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

D:\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
D:\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
D:\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
help(d2l.synthetic_data)

Help on function synthetic_data in module d2l.torch:

synthetic_data(w, b, num_examples)
    Generate y = Xw + b + noise.
    
    Defined in :numref:`sec_linear_scratch`



## 3.3.2 读取数据集

In [4]:
#调用框架中现有的API来读取数据
#将features和labels作为API的参数传递，并通过数据迭代器指定batch_size
#布尔值is_train表示是否希望数据迭代器对象在每个周期内打乱数据

def load_array(data_arrays, batch_size, is_train=True): #@save
    """构造一个Pytorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train) #dataloader是一个数据迭代器

batch_size = 10
#Batch_size是每次喂给模型的样本数量，Epoch_size是训练所有样本总的次数
data_iter = load_array((features, labels), batch_size)

In [5]:
next(iter(data_iter))

[tensor([[ 0.2143,  0.5266],
         [ 2.4342,  0.7887],
         [-1.2598,  0.4435],
         [ 0.5399, -1.1623],
         [-1.8186, -1.3944],
         [-1.4565,  1.1335],
         [ 1.7259, -0.3287],
         [-0.7619,  1.2111],
         [-1.1216,  1.3229],
         [-1.5798, -0.8276]]),
 tensor([[ 2.8349],
         [ 6.3765],
         [ 0.1628],
         [ 9.2276],
         [ 5.2993],
         [-2.5944],
         [ 8.7644],
         [-1.4391],
         [-2.5458],
         [ 3.8601]])]

In [6]:
help(iter)

Help on built-in function iter in module builtins:

iter(...)
    iter(iterable) -> iterator
    iter(callable, sentinel) -> iterator
    
    Get an iterator from an object.  In the first form, the argument must
    supply its own iterator, or be a sequence.
    In the second form, the callable is called until it returns the sentinel.



## 3.3.3 定义模型

In [7]:
#我们首先定义一个模型变量net，它是一个Sequential类的实例。Sequential类将多个层串联在一起。
#当给定输入数据时，Sequential实例将数据传入到第一层，然后将第一层的输出作为第二层的输入，以此类推。

#PyTorch中，全连接层在Linear类中定义。
#我们将两个参数传递到nn.Linear中，第一个指定输入特征情况，即2，第二个指定输出特征形状，输出特征形状为单个标量，因此为1。

#nn是神经网络的缩写
from torch import nn
net = nn.Sequential(nn.Linear(2, 1))

## 3.3.4 初始化模型参数

In [8]:
#在使用net之前我们要初始化模型参数，如在线性回归中的权重和偏置。
#我们指定每个权重参数应该从均值为0、标准差为0.01的正态分布中随机采样，偏置参数将初始化为0。

In [10]:
#我们能直接访问参数以设定它们的初始值。我们通过net[0]选择网络中的第一个图层。
#使用weight.data和bias.data方法访问参数，可以使用替换方法normal_和fill_来重写参数
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

## 3.3.5 定义损失函数

In [11]:
#计算均方误差使用的是MSELoss类，也称为平方L2范数。默认情况下，它返回所有样本损失的平均值。
loss = nn.MSELoss()

## 3.3.6 定义优化算法

In [12]:
#小批量随机梯度下降算法是一种优化神经网络的标准工具，Pytorch在optim模块中实现了该算法的许多变种。
#当我们实例化一个SGD实例时，我们需要指定优化的参数以及优化算法所需要的超参数字典。小批量随机梯度下降只需要设置lr值，这里设置为0.03
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

## 3.3.7 训练

In [13]:
#回顾：在每个迭代周期里我们将完整遍历一次数据集(train_data)，不停地从中获取一个小批量的输入和相应的标签。
#对于每一个小批量会进行以下步骤：1.通过调用net(X)生成预测并计算损失l（前向传播）；
#2.通过反向传播来计算梯度； 3.通过调用优化器来更新模型参数

In [21]:
#为了更好的衡量训练效果，我们计算每个迭代周期后的损失，并打印它来监控训练过程。
num_epoches = 3
for epoch in range(num_epoches):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad() #每次调用SGD之后都要清空，因为我们进行的是小批量梯度下降
        l.backward()
        trainer.step() #step更新所有的参数，进行单次优化
    l = loss(net(features), labels) # 完成一次迭代之后查看我们的损失函数的值
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000094
epoch 2, loss 0.000095
epoch 3, loss 0.000095


In [18]:
help(trainer.step())

Help on NoneType object:

class NoneType(object)
 |  Methods defined here:
 |  
 |  __bool__(self, /)
 |      self != 0
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.



In [22]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([-0.0008, -0.0009])
b的估计误差： tensor([7.1526e-06])


## 3.3.8 小结

In [23]:
#我们可以使用PyTorch的高级API更简洁地实现模型
#在Pytorch中，data模块提供了数据处理工具，nn模块定义了大量的神经网络层和常见损失函数。
#我们可以通过_结尾的方法将参数替换，从而初始化参数。